## Student Practice

From Last Time:

- simple http get request of html from root of a location
- http get request of html from non-root (but default)
- http get request of html from non-root non-default
- request for non-existant resource
- http get request of csv

More advanced HTTP operations:
- http GET request with parameters
- http POST request with parameters
- request for redirected resource
- request for protected resource

## Programmatic HTTP `GET` request

[`requests` module documentation](http://docs.python-requests.org/en/master/)

In [1]:
import requests

In [2]:
protocol = 'http://'
location = 'httpbin.org'
resource = '/get'

template = '{}{}{}'
url = template.format(protocol, location, resource)

url

'http://httpbin.org/get'

We can use a GET to retrieve the specified resource:

In [3]:
resp = requests.get(url)
if resp.status_code != 200:
    print('Error retieving request from', url, "Status:", resp.status_code)
else:
    print(resp.request.path_url)

/get


In this case, the body constains JSON formatted information, as does all of the resources available from `httpbin.org`, and the content of that JSON is designed to reflect back to the client many aspects of the HTTP request.

In [4]:
print(resp.headers['Content-Type'])

application/json


And the nice thing about JSON content is that response objects have a `json()` method and this can be used to interpret the result body as a data structure in our Python program.

In [5]:
result_dictionary = resp.json()
result_dictionary

{'args': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Connection': 'close',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.18.4'},
 'origin': '140.141.243.118',
 'url': 'http://httpbin.org/get'}

### URL request to a JSON that uses URL parameters

Resources that we request of a Data System provider may not always be for an existing file (text or otherwise).  Just like when we make requests to an SQL server and specify (through our SQL SELECT query) some part of the data we are interested in, and the server processes from the stored files and tables and returns just the requested subset, we need such dynamic flexibility in the Data System providers that use HTTP.

This means that we need to pass additional information through HTTP, but still adhere to the methods/verbs that the protocol provides.  One means of passing additional information is to extend the use of our URL to include one or more key-value pairs.  These can be used by the server/provider, which can dynamically create our response.

From the Python programmatic point of view, we can think of the names of our parameters and their values as a dictionary.  The keys give the names of the parameters we are specifying and they map to the values of the arguments.  So if we wanted to specify a parameter of `width` with a value of the integer 500, and a parameter of `label` with a value consisting of the string `"cs181 is fun"`, we could create a dictionary as follows:

In [6]:
urlparams = {'width': 500, 'label': "cs181 is fun"}

And the `requests` module `get()` function has an additional named argument named `params`, which we can use as follows:

In [7]:
resp = requests.get(url, params=urlparams)
if resp.status_code != 200:
    print('Error retieving request from', url, "Status:", resp.status_code)
else:
    print(resp.request.path_url)

/get?width=500&label=cs181+is+fun


See how the additional information is reflected in the JSON returned by `httpbin.org`

In [8]:
result_dictionary = resp.json()
result_dictionary

{'args': {'label': 'cs181 is fun', 'width': '500'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Connection': 'close',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.18.4'},
 'origin': '140.141.243.118',
 'url': 'http://httpbin.org/get?width=500&label=cs181+is+fun'}

But we also need to understand what happens in the HTTP.  In this case, looking at the request path url tells us, excluding the location information, the Resource Identifier specified in the HTTP GET: 

In [9]:
resp.request.path_url

'/get?width=500&label=cs181+is+fun'

Notice what happened to the specification of the resource.  The `/get` had a `?` added, and then one or more `<`key`>`=`<`value`>` pairs separated by the `&` character.  But also note that, since a resource identifier cannot include spaces, the spaces were replace by the `+` character.

**Repeat the sequence starting with the setting of the `urlparams` dictionary at least three times, experimenting with different kinds of values and employing special characters, including some of the characters we see being used to start the query parameters, and to separate key/value pairs.**

## Real World Example of HTTP GET Parameters

Kiva is an organization that serves as a matching service between organizations and individuals from underdeveloped areas of the world with lenders interested in helping such recipients.  Kiva provides HTTP access to allow developers to request various parts of the data they maintain.  Particular subsets of the data are available via resource identifiers known as **endpoints.** . Look at the following link where they document the `/loans/newest` endpoint:
[Kiva loans/newest](https://build.kiva.org/api#GET*|loans|newest)

The cell below sets up the basic url we can use for a GET HTTP request.

In [10]:
protocol = 'https://'
location = 'api.kivaws.org'
resource = '/v1/loans/newest'
fmt = 'json'

template = '{}{}{}.{}'
url = template.format(protocol, location, resource, fmt)

url

'https://api.kivaws.org/v1/loans/newest.json'

From the documentation, try to understand how the API uses GET parameters to customize a request to obtain data.  Then create a dictionary specifying a permutations of the `page`, `per_page`, and the `ids_only` parameters.

In [11]:
loansparams = {'page': 3, 'per_page': 2, 'ids_only': 'false'}

**Now, make `get()` requests and look at how different settings of the parameters yield different results from the data provider.**

In [14]:
resp=requests.get(url, params=loansparams)
print(resp.request.path_url)

/v1/loans/newest.json?page=3&per_page=2&ids_only=false


In [15]:
resp.text

'{"paging":{"page":3,"total":4410,"page_size":2,"pages":2205},"loans":[{"id":1498079,"name":"Al-Dahabia Group","description":{"languages":["en"]},"status":"fundraising","funded_amount":0,"basket_amount":0,"image":{"id":2809332,"template_id":1},"activity":"Personal Medical Expenses","sector":"Health","themes":["Refugees\\/Displaced","Vulnerable Groups"],"use":"to pay for necessary medical surgery expenses she has to undergo.","location":{"country_code":"LB","country":"Lebanon","town":"Qana","geo":{"level":"town","pairs":"33.833333 35.833333","type":"point"}},"partner_id":77,"posted_date":"2018-03-30T14:30:02Z","planned_expiration_date":"2018-04-29T14:30:02Z","loan_amount":2100,"borrower_count":3,"lender_count":0,"bonus_credit_eligibility":true,"tags":[]},{"id":1497458,"name":"Angelica","description":{"languages":["es","en"]},"status":"fundraising","funded_amount":0,"basket_amount":0,"image":{"id":2808391,"template_id":1},"activity":"Agriculture","sector":"Agriculture","use":"to plant a 

In [16]:
resp.content

b'{"paging":{"page":3,"total":4410,"page_size":2,"pages":2205},"loans":[{"id":1498079,"name":"Al-Dahabia Group","description":{"languages":["en"]},"status":"fundraising","funded_amount":0,"basket_amount":0,"image":{"id":2809332,"template_id":1},"activity":"Personal Medical Expenses","sector":"Health","themes":["Refugees\\/Displaced","Vulnerable Groups"],"use":"to pay for necessary medical surgery expenses she has to undergo.","location":{"country_code":"LB","country":"Lebanon","town":"Qana","geo":{"level":"town","pairs":"33.833333 35.833333","type":"point"}},"partner_id":77,"posted_date":"2018-03-30T14:30:02Z","planned_expiration_date":"2018-04-29T14:30:02Z","loan_amount":2100,"borrower_count":3,"lender_count":0,"bonus_credit_eligibility":true,"tags":[]},{"id":1497458,"name":"Angelica","description":{"languages":["es","en"]},"status":"fundraising","funded_amount":0,"basket_amount":0,"image":{"id":2808391,"template_id":1},"activity":"Agriculture","sector":"Agriculture","use":"to plant a

In [17]:
import json

In [20]:
data = json.loads(resp.text)
data

{'loans': [{'activity': 'Personal Medical Expenses',
   'basket_amount': 0,
   'bonus_credit_eligibility': True,
   'borrower_count': 3,
   'description': {'languages': ['en']},
   'funded_amount': 0,
   'id': 1498079,
   'image': {'id': 2809332, 'template_id': 1},
   'lender_count': 0,
   'loan_amount': 2100,
   'location': {'country': 'Lebanon',
    'country_code': 'LB',
    'geo': {'level': 'town', 'pairs': '33.833333 35.833333', 'type': 'point'},
    'town': 'Qana'},
   'name': 'Al-Dahabia Group',
   'partner_id': 77,
   'planned_expiration_date': '2018-04-29T14:30:02Z',
   'posted_date': '2018-03-30T14:30:02Z',
   'sector': 'Health',
   'status': 'fundraising',
   'tags': [],
   'themes': ['Refugees/Displaced', 'Vulnerable Groups'],
   'use': 'to pay for necessary medical surgery expenses she has to undergo.'},
  {'activity': 'Agriculture',
   'basket_amount': 0,
   'bonus_credit_eligibility': False,
   'borrower_count': 1,
   'description': {'languages': ['es', 'en']},
   'funded

## URL POST with Body Parameters

One of the distinguishing features of a POST, as compared to a GET, is that the client requestor includes a **body** part of the message (after the headers and the separating CRLF).  If we are specifying a new version/update of a resource, then the contents of the body contains that data, in whatever form is expected and agreed upon between the client and the server.

POST requests are also used, depending on the design decisions of the server, when we, in a web page, fill out a **form** and then **submit** the form by clicking a button within the page.  The set of name-value pairs from the fields filled in on the form are then placed in the **body** of the POST request.

If we want to use our Python programs to gather data back from a server that might be the result of such a form-based use of a POST, we need to be able to specify the name-value pairs and have them placed in the body, and we need to be able to make a POST request instead of a GET request.

Our `requests` module makes this remarkably simple.  Just like in a GET request where we want to specify URL parameters, and we do it by creating a dictionary, we create a dictionary of the desired POST **body** name-value pairs and use a named parameter in the `requests` `post()` function.  Following is an example that uses the POST endpoint of `httbin.org`:

In [39]:
protocol = 'http://'
location = 'httpbin.org'
resource = '/post'

template = '{}{}{}'
url = template.format(protocol, location, resource)

url

'http://httpbin.org/post'

In [42]:
bodyparams = {'Name': 'Thomas C. Bressoud', 'Phone': '740-555-1212', 'CreditCard': 375412312345}
resp = requests.post(url, data=bodyparams)
if resp.status_code != 200:
    print('Error retieving post request from', url, "Status:", resp.status_code)
else:
    print(resp.request.path_url)
    print(resp.request.body)
    resultD = resp.json()

/post
Name=Thomas+C.+Bressoud&Phone=740-555-1212&CreditCard=375412312345


If called for, we can include **both** params as part of the url and params as part of the body:

In [44]:
urlparams = {'a': 5, 'b': 42}
bodyparams = {'Name': 'Thomas C. Bressoud', 'Phone': '740-555-1212', 'CreditCard': 375412312345}
resp = requests.post(url, params=urlparams, data=bodyparams)
if resp.status_code != 200:
    print('Error retieving post request from', url, "Status:", resp.status_code)
else:
    print(resp.request.path_url)
    print(resp.request.body)
    resultD = resp.json()

/post?a=5&b=42
Name=Thomas+C.+Bressoud&Phone=740-555-1212&CreditCard=375412312345


In [60]:
resultD

{'args': {'a': '5', 'b': '42'},
 'data': '',
 'files': {},
 'form': {'CreditCard': '375412312345',
  'Name': 'Thomas C. Bressoud',
  'Phone': '740-555-1212'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Connection': 'close',
  'Content-Length': '66',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.18.4'},
 'json': None,
 'origin': '140.141.243.118',
 'url': 'http://httpbin.org/post?a=5&b=42'}

**Experiment with different combinations of URL parameters and BODY parameters in POST requests to `httpbin.org`**

## Real World Example of HTTP POST with BODY parameters

Follow with the instructor to CA Gas (margin) prices